<a href="https://colab.research.google.com/github/krvamsi96/Image-classification/blob/master/img_classifier_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install --upgrade tensorflow==2.0.0-rc1

     |████████████████████████████████| 86.3MB 51kB/s 
     |████████████████████████████████| 501kB 61.8MB/s 
     |████████████████████████████████| 4.3MB 54.5MB/s 
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc3


# datasets:- https://drive.google.com/open?id=1SdPVeKG5-V_YBkt-bWlZliFxf8yc-plb



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
train=pd.read_csv("/content/gdrive/My Drive/Dddd/train.csv")
test=pd.read_csv("/content/gdrive/My Drive/Dddd/test.csv")

FileNotFoundError: ignored

In [0]:
train.head()

In [0]:
test.head()

In [0]:
Class_map={'Food':0,'Attire':1,'Decorationandsignage':2,'misc':3}
inverse_map={0:'Food',1:'Attire',2:'Decorationandsignage',3:'misc'}
train['Class']=train['Class'].map(Class_map)

In [0]:
train['Class']

In [0]:
train_img=[]
train_label=[]
j=0
path="/content/gdrive/My Drive/Dddd/Train Images"
for i in tqdm(train['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(150,150))
    img=img.astype('float32')
    train_img.append(img)
    train_label.append(train['Class'][j])
    j=j+1

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
data=ImageDataGenerator(featurewise_center=False,
                        samplewise_center=False,
                        featurewise_std_normalization=False,
                        samplewise_std_normalization=False,
                        zca_whitening=False,
                         rotation_range=10,
                        zoom_range = 0.1,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        horizontal_flip=True,
                        vertical_flip=False)
data.fit(train_img)

In [0]:
test_img=[]
path="/content/gdrive/My Drive/Dddd/Test Images"
for i in tqdm(test['Image']):
    final_path=os.path.join(path,i)
    img=cv2.imread(final_path)
    img=cv2.resize(img,(150,150))
    img=img.astype('float32')
    test_img.append(img)

In [0]:
train_img=np.array(train_img)
test_img=np.array(test_img)
train_label=np.array(train_label)
print(train_img.shape)
print(test_img.shape)
print(train_label.shape)

In [0]:
train_label

In [0]:
train_img=train_img/255.0
test_img=test_img/255.0

In [0]:
test_img

In [0]:
train_img[1]

In [0]:
plt.imshow(train_img[0])
plt.show()

In [0]:
plt.figure(figsize=(15,15))
for i in range(30):
 plt.subplot(6,6,i+1)
 plt.xticks([])
 plt.yticks([])
 plt.grid(False)
 plt.imshow(train_img[i], cmap=plt.cm.binary)
 plt.xlabel(inverse_map[train_label[i]])
plt.show()

In [0]:
plt.figure(figsize=(15,15))
for i in range(30):
 plt.subplot(6,6,i+1)
 plt.xticks([])
 plt.yticks([])
 plt.grid(False)
 plt.imshow(test_img[i], cmap=plt.cm.binary)
 plt.xlabel(inverse_map[train_label[i]])
plt.show()

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [0]:
base_model=VGG16(include_top=False, weights='imagenet',input_shape=(150,150,3), pooling='avg')

model=Sequential()
model.add(base_model)

model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [0]:
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop

base_model.trainable=False

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

In [0]:
callbacks = [reduce_learning_rate]
model.compile( optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(data.flow(train_img, to_categorical(train_label,4), batch_size=32),
                    epochs=10,callbacks=callbacks)

In [0]:
labels = model.predict(test_img)
print(labels[:4])
label = [np.argmax(i) for i in labels]
class_label = [inverse_map[x] for x in label]
print(class_label[:3])
submission = pd.DataFrame({ 'Image': test.Image, 'Class': class_label })
submission.head(10)